In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
import requests 
import pandas_datareader.data as web
from io import StringIO
import yfinance as yf
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

c:\Users\Victor.moreira\AppData\Local\miniconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
### Obtendo a base de dados diária 

base = pd.read_excel('base.xlsx', sheet_name= "Dados Macro Diários")

In [3]:
base.set_index(['Data'], inplace=True)

In [4]:
base

,USD/BRL,CDS,VIX,OIL,USYields_1y,Pré_1y,Diff_Juros,DXY,BCOM,BRZ5y,T10y
Data,,,,,,,,,,,
2022-01-03,5.6818,201.50,16.60,78.98,0.397,11.970,11.527237,96.21,99.4640,10.920,1.637
2022-01-04,5.6770,199.75,16.91,80.00,0.376,12.010,11.590420,96.26,100.5795,11.260,1.649
2022-01-05,5.7087,202.46,19.73,80.80,0.404,12.205,11.753516,96.17,101.1573,11.260,1.700
2022-01-06,5.6834,209.47,19.61,81.99,0.432,12.140,11.657639,96.32,100.7846,11.075,1.728
2022-01-07,5.6355,209.45,18.76,81.75,0.417,12.125,11.659380,95.72,101.2864,11.195,1.765
...,...,...,...,...,...,...,...,...,...,...,...
2025-03-04,5.8852,184.09,23.51,71.04,4.072,14.775,10.284226,105.74,104.4213,14.980,4.279
2025-03-05,5.7394,178.77,21.93,69.30,4.019,14.695,10.263510,104.30,104.4518,14.705,4.279
2025-03-06,5.7639,173.83,24.87,69.46,4.053,14.673,10.206337,104.06,104.7967,14.458,4.304


In [5]:
df_monthly = base.resample("M").agg({
    "USD/BRL": "last",
    "CDS": "last",
    "VIX": "last",
    "OIL": "last",
    "USYields_1y": "last",
    "Pré_1y": "last",
    "Diff_Juros": "last",
    "DXY": "last",
    "BCOM": "last",
    "BRZ5y": "last"
})

C:\Users\Victor.moreira\AppData\Local\Temp\ipykernel_14348\3742197332.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = base.resample("M").agg({


In [6]:
df_monthly1 = base.resample('M').last().pct_change().dropna()

C:\Users\Victor.moreira\AppData\Local\Temp\ipykernel_14348\313154250.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly1 = base.resample('M').last().pct_change().dropna()


In [7]:
df_monthly1

,USD/BRL,CDS,VIX,OIL,USYields_1y,Pré_1y,Diff_Juros,DXY,BCOM,BRZ5y,T10y
Data,,,,,,,,,,,
2022-02-28,-0.027187,-0.038094,0.214257,0.107225,0.318123,0.018519,-0.003614,0.001761,0.062019,0.017234,0.021300
2022-03-31,-0.081571,-0.065729,-0.318076,0.068522,0.590504,0.021739,-0.033379,0.016544,0.086063,0.002229,0.287047
2022-04-30,0.049188,0.074817,0.624514,0.013252,0.279851,0.012379,-0.029921,0.047299,0.040767,0.087633,0.252878
2022-05-31,-0.048390,0.006527,-0.215868,0.123468,-0.002430,0.028277,0.034058,-0.011752,0.014367,0.016769,-0.030293
2022-06-30,0.110895,0.324294,0.096220,-0.065370,0.358987,0.029357,-0.036950,0.028796,-0.108836,0.037007,0.058968
2022-07-31,-0.015753,-0.057736,-0.257053,-0.041808,0.065950,0.000361,-0.017942,0.011655,0.040828,-0.004267,-0.118992
2022-08-31,0.001875,-0.063999,0.212846,-0.122898,0.171150,-0.013353,-0.068386,0.026440,-0.001525,-0.067783,0.202408
2022-09-30,0.044819,0.190293,0.222265,-0.088403,0.147574,-0.009510,-0.067848,0.031463,-0.083463,-0.014626,0.198060
2022-10-31,-0.043635,-0.110125,-0.181531,0.078104,0.160120,-0.021418,-0.102974,-0.005262,0.016724,-0.007634,0.057717


In [8]:
df_monthly1["var_cambio"] = df_monthly["USD/BRL"].pct_change()*100
df_monthly1.dropna(inplace=True)

In [9]:
## Separando as variáveis 

var_internas = ['CDS','Diff_Juros','BRZ5y']
var_externas = ['VIX','OIL','DXY','BCOM']

X = df_monthly1[var_internas + var_externas]
y = df_monthly1['var_cambio']

X = sm.add_constant(X)


In [10]:
model = sm.OLS(y,X).fit()

In [11]:
# Obter as contribuições mensais de cada fator
contributions = X.multiply(model.params, axis=1)


In [12]:
contributions['Fatores Externos'] = contributions[var_externas].sum(axis=1)
contributions['Fatores Internos'] = contributions[var_internas].sum(axis=1)
contributions['Resíduo'] = y - contributions[var_internas + var_externas].sum(axis=1)

In [13]:
contributions[['Fatores Externos', 'Fatores Internos', 'Resíduo']].to_excel('testecontribuicao.xlsx')